In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [3]:
train_data_dir =  '/content/drive/MyDrive/기계학습/train'

In [7]:
img_height = 224
img_width = 224
batch_size = 64

이미지 전처리

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255, #이미지 rescailing
        zoom_range = 0.1,   # 이미지를 랜덤 확대/축소
        horizontal_flip=True, #좌우반전
        brightness_range=[0.2,1.0], # 밝기 조절
        rotation_range=30, # 회전제한 각도 30도
        width_shift_range=0.2, # 좌우이동 20%
        height_shift_range=0.2, # 상하이동 20%
        shear_range=0.15, #shear 연산
        validation_split=0.2)  

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    batch_size = batch_size,
    target_size=(img_height, img_width),
    class_mode='sparse',
    shuffle = True,
    subset='training')

validation_generator = val_datagen.flow_from_directory(
    directory=train_data_dir,
    batch_size=batch_size,
    target_size=(img_height, img_width),
    class_mode='sparse',
    subset='validation')

Found 1998 images belonging to 6 classes.
Found 497 images belonging to 6 classes.


모델 정의

In [11]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers

base_model = tf.keras.applications.resnet_v2.ResNet152V2(weights='imagenet', include_top=False)

# 새로운 분류층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)  # GAP 레이어
predictions = Dense(6, activation='softmax')(x)  #출력층

# 최종 모델
model = Model(inputs=base_model.input, outputs=predictions)

# 새로 추가한 레이어만 학습하도록 설정
for layer in base_model.layers:
    layer.trainable = False


#callback
early= EarlyStopping(patience=5)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/기계학습/model_resnet152v2.h5',
															save_best_only=True,
															monitor='val_loss',
															mode='min',
															verbose=1)

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks = [early, checkpoint]
)

Epoch 1/20
31/31 [==============================] - ETA: 0s - loss: 1.5253 - accuracy: 0.4080 
Epoch 1: val_loss improved from inf to 1.00762, saving model to /content/drive/MyDrive/기계학습/model_resnet152v2.h5
31/31 [==============================] - 492s 15s/step - loss: 1.5253 - accuracy: 0.4080 - val_loss: 1.0076 - val_accuracy: 0.6362
Epoch 2/20
31/31 [==============================] - ETA: 0s - loss: 0.9235 - accuracy: 0.6851
Epoch 2: val_loss improved from 1.00762 to 0.71128, saving model to /content/drive/MyDrive/기계학습/model_resnet152v2.h5
31/31 [==============================] - 91s 3s/step - loss: 0.9235 - accuracy: 0.6851 - val_loss: 0.7113 - val_accuracy: 0.7812
Epoch 3/20
31/31 [==============================] - ETA: 0s - loss: 0.7556 - accuracy: 0.7291
Epoch 3: val_loss improved from 0.71128 to 0.67635, saving model to /content/drive/MyDrive/기계학습/model_resnet152v2.h5
31/31 [==============================] - 97s 3s/step - loss: 0.7556 - accuracy: 0.7291 - val_loss: 0.6764 - va

In [ ]:
history = model.history

# history 객체에서 validation loss와 accuracy 추출
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Epoch 수
epochs = range(1, len(val_loss) + 1)

# Validation Loss 그래프
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Validation Accuracy 그래프
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Fine tuning

In [13]:
for layer in base_model.layers:
  layer.trainable=True

early= EarlyStopping(patience=5)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/기계학습/model_resnet152v2_ft.h5',
															save_best_only=True,
															monitor='val_loss',
															mode='min',
															verbose=1)

optimizer=optimizers.Adam(lr=0.0001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,
              metrics=['accuracy'])


history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks = [early,checkpoint]
)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
31/31 [==============================] - ETA: 0s - loss: 0.4622 - accuracy: 0.8418
Epoch 1: val_loss improved from inf to 0.76139, saving model to /content/drive/MyDrive/기계학습/model_resnet152v2_ft.h5
31/31 [==============================] - 140s 4s/step - loss: 0.4622 - accuracy: 0.8418 - val_loss: 0.7614 - val_accuracy: 0.8438
Epoch 2/100
31/31 [==============================] - ETA: 0s - loss: 0.1905 - accuracy: 0.9374
Epoch 2: val_loss improved from 0.76139 to 0.48015, saving model to /content/drive/MyDrive/기계학습/model_resnet152v2_ft.h5
31/31 [==============================] - 127s 4s/step - loss: 0.1905 - accuracy: 0.9374 - val_loss: 0.4801 - val_accuracy: 0.8951
Epoch 3/100
31/31 [==============================] - ETA: 0s - loss: 0.1559 - accuracy: 0.9473
Epoch 3: val_loss improved from 0.48015 to 0.40202, saving model to /content/drive/MyDrive/기계학습/model_resnet152v2_ft.h5
31/31 [==============================] - 112s 4s/step - loss: 0.1559 - accuracy: 0.9473 - val_loss:

Prediction

In [14]:
base_model = tf.keras.applications.resnet_v2.ResNet152V2(weights='imagenet', include_top=False)

# 새로운 분류기 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)  # GAP 레이어
predictions = Dense(6, activation='softmax')(x)  

# 최종 모델을 구축합니다.
model = Model(inputs=base_model.input, outputs=predictions)

model.load_weights('/content/drive/MyDrive/기계학습/model_resnet152v2_ft.h5')

In [15]:
import os
folder_path = '/content/drive/MyDrive/기계학습/test' 
image_files = os.listdir(folder_path)

In [16]:
sorted_list = sorted(image_files, key=lambda x: int(''.join(filter(str.isdigit, x))))
print(sorted_list)

['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg', '51.jpg', '52.jpg', '53.jpg', '54.jpg', '55.jpg', '56.jpg', '57.jpg', '58.jpg', '59.jpg', '60.jpg', '61.jpg', '62.jpg', '63.jpg', '64.jpg', '65.jpg', '66.jpg', '67.jpg', '68.jpg', '69.jpg', '70.jpg', '71.jpg', '72.jpg', '73.jpg', '74.jpg', '75.jpg', '76.jpg', '77.jpg', '78.jpg', '79.jpg', '80.jpg', '81.jpg', '82.jpg', '83.jpg', '84.jpg', '85.jpg', '86.jpg', '87.jpg', '88.jpg', '89.jpg', '90.jpg', '91.jpg', '92.jpg', '93.jpg', '94.jpg', '95.jpg', '96.jpg', '97.jpg', '98.jpg', '99.jpg', '100.jpg'

In [17]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model

df1 = pd.DataFrame(columns=['Id', 'Category'])
# 폴더 경로 설정
folder_path = '/content/drive/MyDrive/기계학습/test'

# 이미지 크기 설정
img_width, img_height = 224, 224


# 폴더 내 이미지 파일 목록 가져오기
image_files = [file for file in os.listdir(folder_path) if file.endswith('.jpg') or file.endswith('.png')]


for file in sorted_list:
    # 이미지 불러오기
    image_path = os.path.join(folder_path, file)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (img_width, img_height))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)

    # 예측 수행
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)

    df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)
    print(f'Image: {file} - Predicted Class: {predicted_class}')

1/1 [==============================] - 3s 3s/step


<ipython-input-17-36c1590347a7>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)


Image: 0.jpg - Predicted Class: 5
1/1 [==============================] - 0s 43ms/step
Image: 1.jpg - Predicted Class: 0


<ipython-input-17-36c1590347a7>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)


1/1 [==============================] - 0s 34ms/step
Image: 2.jpg - Predicted Class: 5


...

<ipython-input-17-36c1590347a7>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)
<ipython-input-17-36c1590347a7>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)


1/1 [==============================] - 0s 34ms/step
Image: 386.jpg - Predicted Class: 3
1/1 [==============================] - 0s 38ms/step
Image: 387.jpg - Predicted Class: 3


<ipython-input-17-36c1590347a7>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)
<ipython-input-17-36c1590347a7>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)


1/1 [==============================] - 0s 36ms/step
Image: 388.jpg - Predicted Class: 0


<ipython-input-17-36c1590347a7>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'Id': file, 'Category': predicted_class}, ignore_index=True)


In [18]:
label_counts = df1['Category'].value_counts()
print(label_counts)

5    103
0     96
1     80
3     69
2     30
4     11
Name: Category, dtype: int64


In [19]:
df1.to_csv('/content/drive/MyDrive/기계학습/predictions7.csv', index=False)